# Transaction Function P with 6 Actions
This part of the code generate a table with all the transactions. 

In [1]:
import numpy as np
import pandas as pd

# function_p_generator
This class has the function used to generate the table for the Normal and the Tired agent.

In [2]:
class function_p_generator:
    
    def __init__(self,
                 rows=4,
                 cols=9,
                 walls = [
                    ((0,0),(1,0)),((1,0),(0,0)),
                    ((0,1),(1,1)),((1,1),(0,1)),
                    ((0,2),(1,2)),((1,2),(0,2)),
                    ((0,3),(0,4)),((0,4),(0,3)),
                    ((0,4),(0,5)),((0,5),(0,4)),
                    ((0,7),(1,7)),((1,7),(0,7)),
                    ((1,1),(2,1)),((2,1),(1,1)),
                    ((1,2),(2,2)),((2,2),(1,2)),
                    ((1,4),(2,4)),((2,4),(1,4)),
                    ((1,6),(1,7)),((1,7),(1,6)),
                    ((2,0),(2,1)),((2,1),(2,0)),
                    ((2,3),(2,4)),((2,4),(2,3)),
                    ((2,4),(2,5)),((2,5),(2,4)),
                    ((2,6),(2,7)),((2,7),(2,6)),
                    ((2,7),(3,7)),((3,7),(2,7)),
                    ((2,8),(3,8)),((3,8),(2,8)),
                    ((3,3),(3,4)),((3,4),(3,3)),
                    ((3,4),(3,5)),((3,5),(3,4)),
                    ((2,0),(3,0)),((3,0),(2,0)),
                    ((0,4),(1,4)),((1,4),(0,4))],
                beaters = [(2,0), (2,7)],
                pans = [(3,0)],
                ovens = [(3,7)],
                has_beater = [False,True],
                recipes = ['scrumbled','pudding'],
                actions = ['u','d','l','r','p','c']):
        
        self.rows = rows
        self.cols = cols         
        self.walls = walls
        self.beaters = beaters
        self.pans = pans
        self.ovens = ovens
        self.has_beater = has_beater
        self.recipes = recipes
        self.actions = actions
        
    def get_all_states(self):
        states = []
        for re in self.recipes:
            for r in range(self.rows):
                for c in range(self.cols):
                    for b in self.has_beater:
                        states.append((re,r,c,b))
        return states
    

    

    ############################## Normal Agent #######################################
    

    def generate_P_table_function(self,states):
        p_table = {}
        for s in states:
            for s_next in states:
                for a in self.actions:
                    p_table[(s,s_next, a)] = 0
        for s in states:
            for a in self.actions:
                (recipe,row,col, has_beater) = s
                if a == 'l':
                    self.gen_p_table_row(p_table, s, 1,'l')
                if a == 'r':
                    self.gen_p_table_row(p_table, s, 1,'r')
                if a == 'd':
                    self.gen_p_table_row(p_table, s, 1,'d')  
                if a == 'u':
                    self.gen_p_table_row(p_table, s, 1,'u')
                if a == 'p':
                    self.gen_p_table_row(p_table, s, 1,'p')
                if a == 'c':
                    self.gen_p_table_row(p_table, s, 1,'c')                    
        return p_table
    
    
    def gen_p_table_row(self, P, s, probability, action):
        (recipe,row,col, has_beater) = s
    # MOVE LEFT
        if action == 'l':
            if col <= 0 or ((row,col),(row,col-1)) in self.walls:
                P[(s, s, action)] = probability
            elif has_beater:
                s_prime = (recipe, row, col-1, True) 
                P[(s, s_prime, action)] = probability #got the beater
            else:
                s_prime = (recipe, row, col-1, False)
                P[(s, s_prime, action)] = probability
                
     # MOVE RIGHT
        if action == 'r':
            if col >= self.cols-1 or ((row,col),(row,col+1)) in self.walls:
                P[(s, s, action)] = probability
            elif has_beater:
                s_prime = (recipe, row, col+1, True) 
                P[(s, s_prime, action)] = probability #got the beater
            else:
                s_prime = (recipe, row, col+1, False)
                P[(s, s_prime, action)] = probability

      # MOVE UP
        if action == "u":
            if row >= self.rows-1 or ((row,col),(row+1,col)) in self.walls :
                P[(s, s, action)] = probability
            elif has_beater:
                s_prime = (recipe, row+1, col, True) 
                P[(s, s_prime, action)] = probability #got the beater
            else: 
                s_prime = (recipe, row+1, col, False)
                P[(s, s_prime, action)] = probability
      # MOVE DOWN
        
        if action == "d":
            if row <= 0 or ((row,col),(row-1,col)) in self.walls :
                P[(s, s, action)] = probability
            elif has_beater:
                s_prime = (recipe, row-1, col, True) 
                P[(s, s_prime, action)] = probability #got the beater
            else: 
                s_prime = (recipe, row-1, col, False)
                P[(s, s_prime, action)] = probability
      # PICK  
        if action == "p":
            if not has_beater and (row, col) in self.beaters:
                s_prime = (recipe, row, col, True)
                P[(s, s_prime, action)] = probability
            else:
                s_prime = s
                P[(s, s_prime, action)] = probability
      #COOK  
        if action == "c":
            #print("Cook")
            if has_beater and recipe == "scrumbled" and (row, col) in self.pans:
                s_prime = (recipe, row, col, True)
                P[(s, s_prime, action)] = probability
            elif has_beater and recipe == "pudding" and (row, col) in self.ovens:
                s_prime = (recipe, row, col, True)
                P[(s, s_prime, action)] = probability
            else:
                s_prime = s
                P[(s, s_prime, action)] = probability
                
    
       
    
############################## Tired Agent #######################################    
    
    
    def generate_P_tired_table_function(self,states):
        p_tired_table = {}
        for s in states:
            for s_next in states:
                for a in self.actions:
                    p_tired_table[(s,s_next, a)] = 0
        
        for s in states:
            for a in self.actions:
                (recipe,row,col, has_beater) = s
                if a == 'l':
                    self.gen_p_tired_table_row(p_tired_table, s, 'l')
                if a == 'r':
                    self.gen_p_tired_table_row(p_tired_table, s, 'r')
                if a == 'd':
                    self.gen_p_tired_table_row(p_tired_table, s, 'd')  
                if a == 'u':
                    self.gen_p_tired_table_row(p_tired_table, s, 'u')
                if a == 'p':
                    self.gen_p_tired_table_row(p_tired_table, s, 'p')
                if a == 'c':
                    self.gen_p_tired_table_row(p_tired_table, s, 'c') 
        return p_tired_table    
     
    
    def gen_p_tired_table_row(self, P, s, action):
        (recipe,row,col, has_beater) = s
        probability = 0
        #row = 1
        #col = 1
    # MOVE LEFT
        if action == 'l':
            # boundaries and walls
            #go up
            if row+1 >= self.rows or ((row,col),(row+1,col)) in self.walls :
                probability = 0.4
                P[(s, s, action)] = probability
            #has beater
            elif has_beater or (row+1, col) :
                s_prime = (recipe, row+1, col, True) 
                P[(s, s_prime, action)] = 0.4 #got the beater
            else: 
                s_prime = (recipe, row+1, col, False)
                P[(s, s_prime, action)] = 0.4
            # boundaries and walls
            #go down
            if row-1 < 0 or ((row,col),(row-1,col)) in self.walls :
                probability = probability + 0.6
                P[(s, s, action)] = probability 
            #beater   
            elif has_beater or (row-1, col):
                s_prime = (recipe, row-1, col, True) 
                P[(s, s_prime, action)] = 0.6 #got the beater
            #no beater down 
            else: 
                s_prime = (recipe, row-1, col, False)
                P[(s, s_prime, action)] = 0.6  

        # MOVE RIGHT
        if action == 'r':
            # boundaries and walls
            #go up
            if row+1 >= self.rows or ((row,col),(row+1,col)) in self.walls :
                probability = 0.6
                P[(s, s, action)] = probability
            #has beater
            elif has_beater or (row+1, col):
                s_prime = (recipe, row+1, col, True) 
                P[(s, s_prime, action)] = 0.6 #got the beater
            else: 
                s_prime = (recipe, row+1, col, False)
                P[(s, s_prime, action)] = 0.6
            # boundaries and walls
            #go down
            if row-1 < 0 or ((row,col),(row-1,col)) in self.walls :
                probability = probability + 0.4
                P[(s, s, action)] = probability 
            #has beater   
            elif has_beater :
                s_prime = (recipe, row-1, col, True) 
                P[(s, s_prime, action)] = 0.4 #got the beater
            #no beater down 
            else: 
                s_prime = (recipe, row-1, col, False)
                P[(s, s_prime, action)] = 0.4  

        # MOVE UP
        if action == 'u':
            #go right
            # boundaries and walls
            if col+1 >= self.cols or ((row,col),(row,col+1)) in self.walls:
                probability = 0.4
                P[(s, s, action)] = probability
            #has beater
            elif has_beater:
                s_prime = (recipe, row, col+1, True) 
                P[(s, s_prime, action)] = 0.4 #got the beater
            else: 
                s_prime = (recipe, row, col+1, False)
                P[(s, s_prime, action)] = 0.4
            # boundaries and walls
            if col-1 <= 0 or ((row,col),(row,col-1)) in self.walls:
                probability = probability + 0.6
                P[(s, s, action)] = probability 
            #has beater    
            elif has_beater:
                s_prime = (recipe, row, col-1, True) 
                P[(s, s_prime, action)] = 0.6 #got the beater
            #no beater down 
            else: 
                s_prime = (recipe, row, col-1, False)
                P[(s, s_prime, action)] = 0.6 
                
        # MOVE DOWN
        if action == 'd':
            #go right
            # boundaries and walls
            if col+1 >= self.cols or ((row,col),(row,col+1)) in self.walls:
                probability = 0.6
                P[(s, s, action)] = probability
            #has beater
            elif has_beater:
                s_prime = (recipe, row, col+1, True) 
                P[(s, s_prime, action)] = 0.6 #got the beater
            else: 
                s_prime = (recipe, row, col+1, False)
                P[(s, s_prime, action)] = 0.6
            # boundaries and walls
            #go left
            if col-1 <= 0 or ((row,col),(row,col-1)) in self.walls:
                probability = probability + 0.4
                P[(s, s, action)] = probability 
            #has beater   
            elif has_beater:
                s_prime = (recipe, row, col-1, True) 
                P[(s, s_prime, action)] = 0.4 #got the beater
            #no beater down 
            else: 
                s_prime = (recipe, row, col-1, False)
                P[(s, s_prime, action)] = 0.4 
      # PICK  
        if action == "p":
            if not has_beater and (row, col) in self.beaters:
                s_prime = (recipe, row, col, True)
                probability = 1
                P[(s, s_prime, action)] = probability
            else:
                s_prime = s
                probability = 1
                P[(s, s_prime, action)] = probability
      #COOK  
        if action == "c":
            #print("Cook")
            if has_beater and recipe == "scrumbled" and (row, col) in self.pans:
                probability = 1
                s_prime = (recipe, row, col, True)
                P[(s, s_prime, action)] = probability
            elif has_beater and recipe == "pudding" and (row, col) in self.ovens:
                probability = 1
                s_prime = (recipe, row, col, True)
                P[(s, s_prime, action)] = probability
            else:
                probability = 1
                s_prime = s
                P[(s, s_prime, action)] = probability                
                
                


In [3]:
generator = function_p_generator()

In [4]:
states = generator.get_all_states()  

# Number of the states

In [5]:
len(states)

144

In [6]:
P = generator.generate_P_table_function(states) 

# Dimensionality of the Transition function P
### the dimension of the transition function p can be calculated as follow:
$$ |P| = |S|x|S|x|A| $$
$$ |P| = 144*144*6 = 124.416 $$

In [7]:
len(P)

124416

In [8]:
P

{(('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'u'): 1,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'd'): 1,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'l'): 1,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'r'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'p'): 1,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'c'): 1,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'u'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'd'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'l'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'r'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'p'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'c'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 1, False), 'u'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 1, False), 'd'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 1, False), 'l'): 0,
 

# The P transition function for the normal agent is saved in the "p_table_6_actions.csv" file

In [9]:
    s = pd.Series(P, name='p_table')
    s.rename_axis((['s', 's_prime', 'a']), inplace=True)
    s.to_csv('p_table_6_actions.csv')

In [10]:
p_table_tired = generator.generate_P_tired_table_function(states)

In [11]:
len(p_table_tired)

124416

# Tired behaviour
#### In the case the agent is tired there are some particular case study:
#### if the agent is in the posion row = 0 and col = 0 he has two wall one up and one down so if he choose the action "Move RIGHT" he go UP with p= 0.6 and DOWN with p = 0.4 but up and down he is sorrounded by walls so the result is that he can change state with p = 1

(('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'r'): 1.0,

#### if in the same position the agent choose the Action = "MOVE UP" the agent has 3 wall near him, one up, one down and the left edge of the board so the result must be the following:

#### with p = 0.6  the agent dont move
(('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'u'): 0.6

#### and with p = 0.4 the angent move perpendicular right of the action choosed

(('scrumbled', 0, 0, False), ('scrumbled', 0, 1, False), 'u'): 0.4,

#### The probability to perform an actions of Pick and Cook don't change in this new scenario. 


In [12]:
p_table_tired

{(('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'u'): 0.6,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'd'): 0.4,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'l'): 1.0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'r'): 1.0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'p'): 1,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, False), 'c'): 1,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'u'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'd'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'l'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'r'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'p'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 0, True), 'c'): 0,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 1, False), 'u'): 0.4,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 1, False), 'd'): 0.6,
 (('scrumbled', 0, 0, False), ('scrumbled', 0, 1, False)

# The P transition function for the tired agent is saved in the "tired_6_actions.csv" file

In [13]:
    s_tired = pd.Series(p_table_tired, name='p_table_tired')
    s_tired.rename_axis((['s', 's_prime', 'a']), inplace=True)
    s_tired.to_csv('tired_6_actions.csv')